In [2]:
import pandas as pd
from pdfminer.pdfparser import PDFParser # 文档分析器
from pdfminer.pdfdocument import PDFDocument # PDF文档对象存处文档结构
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from PyPDF2 import PdfFileReader
from PyPDF2 import PdfFileReader, PdfFileWriter
import pdfplumber
import sys
import importlib
importlib.reload(sys)
import re
import string
import os


年报已爬取，现进行处理，以深交所为例

————————————————————————————————————
### sz目录文件处理

In [4]:
'''
文件已保存
# 新建的年报的清单的地址
path_catlog_sz_2018_2019 = 'F:\zzqaq\\catlog\\catalog_sz_2018_2019.csv'  
pd_catlog_sz_2018_2019 = pd.read_csv(path_catlog_sz_2018_2019,encoding="gbk",converters={'code':str,'year':str})  
pd_catlog_sz_2018_2019["code"] = [str(i).replace("\t","") for i in pd_catlog_sz_2018_2019["code"]]  

# # 所有年报所在文件夹  
path_sz_annual_report_2018_2019 = 'F:\\zzqaq\\sz_annual_report_2018_2019  
'''

In [4]:
def catalog_process(path_catalog):
    # 读取目录
    pdfcatalog_2021_x = pd.read_csv(path_catalog,encoding="gbk",header=None)
    # 换一个列名    
    pdfcatalog_2021_x.rename(columns={0:'pdfname'},inplace=True)
    # 去除文件名包含“英文”的行    
    pdfcatalog_2021_x= pdfcatalog_2021_x[-pdfcatalog_2021_x['pdfname'].str.contains('英文版')]
    # 去除文件名包含“更正公告”的行    
    pdfcatalog_2021_x= pdfcatalog_2021_x[-pdfcatalog_2021_x['pdfname'].str.contains('公告|说明')]
    
    # 提取股票代码
    pdfcatalog_2021_x['code'] = pdfcatalog_2021_x['pdfname'].map(lambda x:x[:6])
    # 排序
    pdfcatalog_2021_x.sort_values(by="code" , inplace=True, ascending=True)
    
    # 去除重复
    pdfcatalog_2021_x.drop_duplicates(subset = ['pdfname'],keep='first',inplace=True) 

    # 重置序号
    pdfcatalog_2021_x = pdfcatalog_2021_x.reset_index(drop=True)
    # 删第二列
    pdfcatalog_2021_x = pdfcatalog_2021_x.drop(1,axis = 1)
    # 沪市：sh;沪市主板：shmb；深市：szse；深主板：szmb；中小板：szzx；创业板：szcy
    return pdfcatalog_2021_x

In [5]:
# 年报没有一次性爬完，需进行合并
pd_catalog_sz_1819_1 = 'F:\\zzqaq\\sz_annual_report_2018_2019_20190101-20201231.csv'
pd_catalog_sz_1819_2 = 'F:\\zzqaq\\catlog\\sz_annual_report_2018_20190101-20191231.csv'
pd_catalog_sz_1819_3 = 'F:\\zzqaq\\catlog\\sz_annual_report_2019_20200101-20201231.csv'

pdfcatalog_sz_1819_1= catalog_process(pd_catalog_sz_1819_1) #936
pdfcatalog_sz_1819_2= catalog_process(pd_catalog_sz_1819_2) #722
pdfcatalog_sz_1819_3= catalog_process(pd_catalog_sz_1819_3) #459
pd_catlog_sz_2018_2019 = pd.concat([pdfcatalog_sz_1819_1, pdfcatalog_sz_1819_2, pdfcatalog_sz_1819_3])

# 按code排序
pd_catlog_sz_2018_2019 = pd_catlog_sz_2018_2019.sort_values('code', ascending=True)
pd_catlog_sz_2018_2019 = pd_catlog_sz_2018_2019.reset_index(drop=True)

# 添加年份
pd_catlog_sz_2018_2019.loc[pd_catlog_sz_2018_2019['pdfname'].str.contains('2020'),'year']='2020'
pd_catlog_sz_2018_2019.loc[pd_catlog_sz_2018_2019['pdfname'].str.contains('2019'),'year']='2019'
pd_catlog_sz_2018_2019.loc[pd_catlog_sz_2018_2019['pdfname'].str.contains('2018'),'year']='2018'
pd_catlog_sz_2018_2019.dropna(subset=['year'], inplace=True)

# # 去重
pd_catlog_sz_2018_2019.drop_duplicates(subset=['code', 'year'], keep='first', inplace=True)
pd_catlog_sz_2018_2019.reset_index(drop=True, inplace=True)

pd_catlog_sz_2018_2019['board'] = pd_catlog_sz_2018_2019['code'].map(lambda x:x[:3])

pd_catlog_sz_2018_2019.head(3)

,pdfname,code,year,board
0,000004国农科技2018年年度报告.(2328k).PDF,000004,2018,000
1,000005世纪星源2019年年度报告.(3118k).PDF,000005,2019,000
2,000005世纪星源2018年年度报告(更新后).(2791k).PDF,000005,2018,000


In [13]:
# 查看重复行
duplicated_rows = pd_catlog_sz_2018_2019[pd_catlog_sz_2018_2019.duplicated(subset=['code'], keep=False)]
print(duplicated_rows)
# 挑出来查看
specific_rows = pd_catlog_sz_2018_2019.loc[pd_catlog_sz_2018_2019['code'] == '603976']
print(specific_rows)

                              pdfname    code  year board
2698  603976正川股份2019年年度报告.(2535k).PDF  603976  2019   603
2699  603976正川股份2018年年度报告.(2157k).PDF  603976  2018   603


In [14]:
# 保存
pd_catlog_sz_2018_2019 = pd_catlog_sz_2018_2019.sort_values('code', ascending=True) # # 按code排序
pd_catlog_sz_2018_2019 = pd_catlog_sz_2018_2019.reset_index(drop=True) # 重置序号
pd_catlog_sz_2018_2019['code'] = '\t'+pd_catlog_sz_2018_2019['code'] # 保存时保留股票代码开头的0
pd_catlog_sz_2018_2019.to_csv('F:\zzqaq\\catlog\\catalog_sz_2018_2019.csv',encoding="gbk",index = False)

In [19]:
# 地址
# 新建的年报的清单的地址
path_catlog_sz_2018_2019 = 'F:\\zzqaq\\catlog\\catalog_sz_2018_2019.csv'
# 读取
pd_catlog_sz_2018_2019 = pd.read_csv(path_catlog_sz_2018_2019,encoding="gbk",converters={'code':str,'year':str})
pd_catlog_sz_2018_2019["code"] = [str(i).replace("\t","") for i in pd_catlog_sz_2018_2019["code"]]

# # 所有年报所在文件夹
path_sz_annual_report_2018_2019 = 'F:\\zzqaq\\sz_annual_report_2018_2019'

szcy市目录文件处理，文件已保存  
——————————————————————————————————————

————————————————————————————————————
### text截取  

In [20]:
# 新建的年报清单的地址
path_catlog_sz_2018_2019 = 'F:\\zzqaq\\catlog\\catalog_sz_2018_2019.csv'
pd_catlog_sz_2018_2019 = pd.read_csv(path_catlog_sz_2018_2019,encoding="gbk",converters={'code':str,'year':str})
pd_catlog_sz_2018_2019["code"] = [str(i).replace("\t","") for i in pd_catlog_sz_2018_2019["code"]]

# # 所有年报所在文件夹
path_sz_annual_report_2018_2019 = 'F:\\zzqaq\\sz_annual_report_2018_2019'

In [21]:
# 从pdf跳转目录获得特定章节页码
def get_page_from_Outlines(pdf_path, key_word):
#     输入单个pdf文件地址、目录关键词
#     输出起始页（已减1）、中止页

    with open(pdf_path, 'rb') as pdf_file:  # 读取pdf文档
        pdf = PdfFileReader(pdf_file)  # 加载pdf文档
        if pdf.isEncrypted:
            pdf.decrypt('')  # 解密
        end_page_number = pdf.getNumPages()  # 获取总页码
        pdf_directory = pdf.getOutlines()  # 获取目录

        is_have_start_page_number = False
        for destination in pdf_directory:
            if isinstance(destination, dict): # 少这个报错
                if is_have_start_page_number: 
                    end_page_number = pdf.getDestinationPageNumber(destination)
                    is_get_page_number_range = True
                    break

                title = destination.get('/Title')
                if key_word in str(title):
                    # 在目录中找到关键词
                    start_page_number = pdf.getDestinationPageNumber(destination)
                    is_have_start_page_number = True
                    continue
    return start_page_number,end_page_number,pdf.getNumPages()

In [22]:
# 不能跳转，提取目录页，根据关键词确定页码
def path_report(pdfcatalog_2021,path_annual_report_2021,i):
    # 输入 年报目录的表，年报文件夹，第i个年报的地址
    # 得到第i个年报的地址，前几页年报的文件名
    
    list_company_name_2021 = pdfcatalog_2021.pdfname   # 公司报表名的 列表
    path_pdf = path_annual_report_2021+'\\'+ list_company_name_2021[i]  # 列表上第i个报表的位置
    pdfname_code_2021 = list_company_name_2021[i][0:6] +'_catalog'   # 用名字前6位数字股票代码重命名一个pdf
    # 例子：'600532_catalog'

    return path_pdf, pdfname_code_2021


# ①截取’目录页‘，生成pdf
def select_page_pdf(start_page,end_page,path,pdfname_code_2021):
#     开始页，结束页，报表的位置，新生成pdf的名字
    output = PdfFileWriter() # 创建一个空的PDF文件写入对象 output
    pdf_file = PdfFileReader(path) # 函数读取PDF文件，将其赋值给 pdf_file 对象。
    pdf_pages_len = pdf_file.getNumPages()
    for i in range(start_page, end_page):
        output.addPage(pdf_file.getPage(i))
    outputStream = open('F:\\zzqaq\\report_discuss_2018_\\discus\\'+pdfname_code_2021+".pdf", "wb")
    output.write(outputStream)

# ②功能：将’目录页pdf‘转为’文本‘，输入目录页的地址
# 重点：不替换数字，不替换'。'或'.'
def pdftotxt(path):
    with open(path, 'rb') as f:
        parser = PDFParser(f) # 传文件进parser而不是路径
        document = PDFDocument(parser)
    #     判断文件是否允许文本提取
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            resmag = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(resmag,laparams = laparams)
            interpreter = PDFPageInterpreter(resmag,device)
    #         处理每一页
            text_all = ''
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                layout = device.get_result()
                text=''
                for y in layout:
                    if (isinstance(y, LTTextBoxHorizontal)):
                        text = text + y.get_text()
                text = text.replace('\n','') # 去掉换行的符号
                for ch in "!#$%&（）*+，\、-/:;<=>?@[\\]_'{|}~'”‘’“,":
                    text = text.replace(ch,'') 
                text = text.replace(" ","")
                text_all = text_all + text
    f.close()
    return text_all


# 获取“管理层讨论与分析”页码
# 从“讨论与分析”定位
loca_str1 = '讨论与分析.' # 定位字符
loca_str2 = '第x节重要事项'
def get_page_2(text_all,loca_str1,loca_str2):

    location_ = text_all.find(loca_str1) #
    counter = 0
    for i in text_all[location_+len(loca_str1):]:
        counter+=1
        if i.isdigit():
            page1 = text_all[location_+len(loca_str1)+counter-1:location_+len(loca_str1)+counter+1]
#             print(page1)
            break

    # 下一个定位字符，从page1的第一个数字的前一个点 到后9位字符，例子：'.20第四节重要事项.'，主要
    loca_str_next = text_all[location_+len(loca_str1)+counter-2:location_+len(loca_str1)+counter+len(loca_str2)+2]
    
    find_location_2 = text_all.find(loca_str_next)
    counter_2 = 0
    for i in text_all[find_location_2+len(loca_str2)+2:]:
        counter_2+=1
        if i.isdigit():
            page2 = text_all[find_location_2+len(loca_str2)+2+counter_2-1:find_location_2+len(loca_str2)+2+counter_2+2]
            break
            
    # 提取数字，防止出现因页码个位数而纳入汉字情况，例如page1 = '和7'
    page1 = [int(s) for s in re.findall(r'-?\d+\.?\d*', page1)][0]
    page2 = [int(s) for s in re.findall(r'-?\d+\.?\d*', page2)][0]
    if page2<=page1: return False
    else: return page1-1,page2 # 管理层讨论与分析当页、重要事项当页


# 对’讨论与分析‘文本做处理，提取函数进行调整，保留逗号、句号，去除数字
def pdftotxt_adjust(path):
    import string
    punctuation_string = string.punctuation # 英文标点
    with open(path, 'rb') as f:
        parser = PDFParser(f)
        document = PDFDocument(parser)
    #     判断文件是否允许文本提取
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            resmag = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(resmag,laparams = laparams)
            interpreter = PDFPageInterpreter(resmag,device)
    #         处理每一页
            text_all = ''
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                layout = device.get_result()
                text=''
                for y in layout:
                    if (isinstance(y, LTTextBoxHorizontal)):
                        text = text + y.get_text()
                text = text.replace('\n','') # 去掉换行的符号
    #             去所有英文标点
                for ch in punctuation_string+'（）“”、：《》':
                    text = text.replace(ch,'') 
                text = re.sub(r'[0-9]+','',text) # 数字替换无
                text = text.replace(" ","")
                text_all = text_all + text
        f.close()
    return text_all

In [30]:
# 新建的年报的清单的地址
path_catlog_sz_2018_2019 = 'F:\\zzqaq\\catlog\\catalog_sz_2018_2019.csv'
pd_catlog_sz_2018_2019 = pd.read_csv(path_catlog_sz_2018_2019,encoding="gbk",converters={'code':str,'year':str})
pd_catlog_sz_2018_2019["code"] = [str(i).replace("\t","") for i in pd_catlog_sz_2018_2019["code"]]

# 所有年报所在文件夹
path_sz_annual_report_2018_2019 = 'F:\\zzqaq\\sz_annual_report_2018_2019'

# 文本提取
def text_extract(pd_catlog_sh_2018_2019,path_sh_annual_report_2018_2019,i):
#     输入目录文档、所有年报的位置

    # pdf文件地址、目录名
    path_pdf, code_catalog = path_report(pd_catlog_sh_2018_2019,path_sh_annual_report_2018_2019,i)
    try:
        #     找页码
        key_word = '讨论与分析'
        page1,page2, page_of_all = get_page_from_Outlines(path_pdf,key_word)
        if page2<page_of_all and (page1!=9 and page2 != 26):True
        else:
            select_page_pdf(2,7,path_pdf,code_catalog) # 目录pdf生成
            # 目录页位置
            path_code_catalog = 'F:\\zzqaq\\report_discuss_2018_\\discus\\'+code_catalog+'.pdf'
            text_all = pdftotxt(path_code_catalog) # 目录页内容转为文本
            loca_str1,loca_str2='讨论与分析.','第x节重要事项'
            page1,page2 = get_page_2(text_all,loca_str1,loca_str2) 
    except:
        select_page_pdf(2,7,path_pdf,code_catalog) # 目录pdf生成
            # 目录页位置
        path_code_catalog = 'F:\\zzqaq\\report_discuss_2018_\\discus\\'+code_catalog+'.pdf'
        text_all = pdftotxt(path_code_catalog) # 目录页内容转为文本
        loca_str1,loca_str2='讨论与分析.','第x节重要事项'
        page1,page2 = get_page_2(text_all,loca_str1,loca_str2) 

    select_page_pdf(page1,page2,path_pdf,code_catalog+"_discussion") # 文本pdf生成
    path_pdf_text = 'F:\\zzqaq\\report_discuss_2018_\\discus\\'+code_catalog+'_discussion.pdf'
    text = pdftotxt_adjust(path_pdf_text)
    
    return pd_catlog_sh_2018_2019.code[i],text,page1,page2

In [24]:
# 例子1，包括目录页文档文本内容、起止页码、讨论与分析章节的文本内容
# pdf文件地址、目录名
path_pdf, code_catalog = path_report(pd_catlog_sz_2018_2019,path_sz_annual_report_2018_2019,10)
# ('F:\\zzqaq\\sh_annual_report_2018_2019/600008首创股份2019年年度报告.(4256k).PDF','600008_catalog')
print('pdf文件地址、目录名',path_pdf, code_catalog)
print('-'*50)

# 生成目录页文档
select_page_pdf(2,7,path_pdf,code_catalog)

# 目录页转文本
path_code_catalog = 'F:\\zzqaq\\report_discuss_2018_\\discus\\'+code_catalog+'.pdf'
text_all = pdftotxt(path_code_catalog)
print('目录文本：',text_all)
print('-'*50)

loca_str1,loca_str2='讨论与分析.','第x节重要事项'
page1,page2 = get_page_2(text_all,loca_str1,loca_str2)
print('文档起始页码:',page1,page2)
print('-'*50)

# 生成“管理层讨论与分析”文档
select_page_pdf(page1,page2,path_pdf,code_catalog+"_discussion")
path_pdf_text = 'F:\\zzqaq\\report_discuss_2018_\\discus\\'+code_catalog+'_discussion.pdf'
text_example = pdftotxt_adjust(path_pdf_text)
text_example

pdf文件地址、目录名 F:\zzqaq\sz_annual_report_2018_2019\000019深粮控股2018年年度报告.(4616k).PDF 000019_catalog
--------------------------------------------------
目录文本： 深圳市深粮控股股份有限公司2018年年度报告全文目录第一节重要提示目录和释义..................................................2第二节公司简介和主要财务指标................................................5第三节公司业务概要.........................................................10第四节经营情况讨论与分析...................................................13第五节重要事项............................................................29第六节股份变动及股东情况...................................................63第七节优先股相关情况.......................................................69第八节董事监事高级管理人员和员工情况...................................70第九节公司治理............................................................76第十节公司债券相关情况.....................................................81第十一节财务报告...........................................................82第十二节备查文件目录......................................................2013深圳市深粮控股股份有限公司2018年年度报告全文释义释义项指释义内容深

'深圳市深粮控股股份有限公司年年度报告全文第四节经营情况讨论与分析一概述报告期内，深粮控股认真执行落实年度重点工作和战略规划目标，实施重大资产重组，收购深粮集团股权，实现地方国有粮食企业整体上市。公司主营业务在茶及天然植物精深加工为主的食品原料配料生产研发和销售的基础上增加粮油储备粮油贸易粮油加工等粮油流通及粮油储备服务业务，公司转型进入双主业齐头并进发展轨道。报告期内，深粮控股围绕打造粮食供应链优质服务商安全优质食品提供商的目标，以持续创新发展为主线，坚持质量引领创新驱动。全年经营效益持续向好，荣膺中国服务业企业强下属深粮集团成功入选国务院国企改革双百行动企业名单下属婺源聚芳永获得国家高新技术企业认定。主要业务发展深粮控股作为深圳市的粮安工程和米袋子，报告期内公司保质保量完成政府储备服务，月均储备粮达万吨，储备油达万吨，通过粮油的均衡轮换保障深圳市粮油供应充足和价格平稳。报告期内，公司立足自身优势和产业发展，运用信息化手段，创新开拓粮油产品供应渠道和交易方式，构建面向家庭类客户，学校写字楼大型企事业机关类客户，以及饲料厂工厂类客户的三位一体食品服务网络，扩大粮油的有效供给和中高端供给，满足人民群众优质多样营养健康绿色便利需要，实现粮食供给从吃得饱到吃得好的转变。一是旗下多喜米网线上会员线下无人零售模式，会员已超万个家庭，年收入突破亿元；二是旗下贝格厨房公司为高端写字楼学校医院企事业机关餐饮等单位提供一站式厨房配送服务，常年服务人口超过万人；三是旗下线上大宗粮油交易平台拓展加工原料撮合服务业务，年成交量突破万吨，成交金额突破亿元。报告期内，下属深粮质检通过了农产品CATL资质认证，取得了CMA证书，全年检测样品达个，同比增加，对外部客户检测样品个。省级粮油库存检查，东北区域储粮质量达标率为，市级粮油库存检查质量达标率为。另外，军粮服务满意度测评的结果为满意，全年投诉。重点项目报告期内，公司东莞粮食物流节点项目同时进行万吨CDE筒仓食品深加工项目一期码头等项目建设，CDE仓项目桩基工程进度达，食品深加工项目主体建设基本完工，国际食品码头实现万吨靠泊能力升级，全年装卸船只条，完成中转量超万吨。获得了广东省重点农业龙头企业全国放心粮油示范仓储企业等多项荣誉。报告期内，中国粮食交易网全面上线平台交易，于年月份开展了首次麸皮竞价交易会，全年共进行场竞价交易会。另外，中国粮食交

In [32]:
# 例子2
code,text,page1,page2 = text_extract(pd_catlog_sz_2018_2019,path_sz_annual_report_2018_2019,66) 
code,text,page1,page2

('000425',
 '徐工集团工程机械股份有限公司年度报告全文第四节经营情况讨论与分析一概述年我国国内生产总值万亿元，同比增长。全国固定资产投资不含农户亿元，同比增长。报告期内，工程机械行业高速增长，整体收入规模盈利能力资产质量大幅提升。数据来源国家统计局网站一年主要经营指标完成情况年，公司实现营业总收入亿元，同比增长，其中，实现海外收入亿元，同比增长；归属于母公司所有者的净利润亿元，同比增长。年，公司持续优化运营指标，经营性现金流净额为亿元，创历史最高水平；加权平均净资产收益率同比提高个百分点，达到，全力夯实了高质量高效益高效率可持续三高一可的高质量发展根基。二年主要工作完成情况年，遵照习近平总书记视察徐工重要指示精神，在一二三三四四战略经营指导思想体系指引下，公司真正凝心聚力大战决战，主要指标再创历史新高，三高一可高质量发展全面推进。在对市场环境的分析应对对瓶颈短板的坚实突破对打赢必胜的坚决策划执行落地，在体系和能力方面都上了一个大台阶。公司过去一年工作主要体现为六个新第一，主要经营指标高质量再创历史新高。在营业总收入同比增长之下，存货同比增长；研发投入同比增长；利息保障倍数同比增长；公司经营的基础指标夯实向好。强控风险一号工程全口径逾期应收账款比年初下降；降本增效一号工程集采和主机厂自采分别净降本亿亿元。持续补备件业务短板，实现备件收入同比增长。公司高质量的经营表现支撑母公司营业收入连续过千亿并呈现高质量的千亿内涵变化运营质量盈利能力资产效率规模结构指标全面压实；风控指标持续强力压降，结构性改革调整转型强内功，各项管理基础持续夯实；是在一二三三四四战略经营指导思想体系技术领先用不毁，做成工艺品产品理念三高一大产品战略以用户为中心经营思维及全面发力高端产品创新全面强化一带一路等国际化市场开拓支撑下的过千亿；是主营占比大幅提高，创新能力国际化能力队伍能力质量效益水平核徐工集团工程机械股份有限公司年度报告全文心竞争力都有大幅提升的过千亿，是厚积薄发高质量的过千亿。第二，全线板块都有高质量发展新业绩。①三大主机事业部引领发力。移动起重机市场占有率稳固保持世界第一位，吨超大吨位全地面起重机市场占有率；吨以上履带吊国内市场占有率增长个百分点，市场占有率突破，稳居行业第一。装载机市场占有率提升个百分点；吨装载机销量同比增长；重庆基地小装销量过千台，加紧规划发展电动叉车与

In [1]:
# 提取出的文档写入csv
import csv
with open("F:\\zzqaq\\data\\text_sz_2018_2019_x.csv",'a',newline='',encoding="gb18030") as f:
    writer = csv.writer(f)
    # 写入列名
    writer.writerow(['code', 'year', 'text'])
    error_i,error_code = [],[]
    for i in range(pd_catlog_sz_2018_2019.shape[0]):
#     for i in range(3,6):
        try:
            code,text,page1,page2 = text_extract(pd_catlog_sz_2018_2019,path_sz_annual_report_2018_2019,i)
            writer.writerow([code, pd_catlog_sz_2018_2019.year[i], text])
            print('成功写入',i)
        except:
            writer.writerow(['\t'+code, '', ''])
            print('出错:',i,pd_catlog_sz_2018_2019.code[i])
            error_i.append(i);error_code.append(pd_catlog_sz_2018_2019.code[i])
            pass
        continue

In [6]:
# 查看
path='F:\\zzqaq\\data\\text_sz_2018_2019_x.csv'
text = pd.read_csv(path,encoding="gb18030",converters={'code':str, 'year':str})
text["code"] = [str(i).replace("\t","") for i in text["code"]]
text.head(3)

,code,year,text
0,000004,2018,深圳中国农大科技股份有限公司年年度报告全文第四节经营情况讨论与分析一概述年度，公司实现主营业...
1,000005,2019,深圳世纪星源股份有限公司年年度报告全文第四节经营情况讨论与分析一概述在本报告期内，本司主营业...
2,000005,2018,深圳世纪星源股份有限公司年年度报告全文第四节经营情况讨论与分析一概述在本报告期内，本司主营业...


深市text截取  
————————————————————————————————————————————

—————————————————————————————————————————————  
### 文档合并

In [63]:
# 读取之前提取的2020年文档并加标签
path='F:\\zzqaq\\sz_report_discussion_2020\\sz_text2020.csv'
pd_sz_text_2020 = pd.read_csv(path,encoding="gb18030",header = None)

pd_sz_text_2020['code'] = pd_sz_text_2020[0].str.slice(stop=6)
pd_sz_text_2020['year']='2020'
pd_sz_text_2020['text'] = pd_sz_text_2020[0].str.slice(start=6)
pd_sz_text_2020 = pd_sz_text_2020.drop(0, axis=1)

# 和之后提取的18、19年文本合并
text_sz_2018_2020 = pd.concat([text, pd_sz_text_2020])

# 按code排序
text_sz_2018_2020 = text_sz_2018_2020.sort_values('code', ascending=True)
text_sz_2018_2020 = text_sz_2018_2020.reset_index(drop=True)

# 去重
text_sz_2018_2020.drop_duplicates(subset=['code', 'year'], keep='first', inplace=True)

# 保存
text_sz_2018_2020.to_csv('F:\\zzqaq\\data\\text_sz_2018_2020.csv',encoding="gb18030",index = False)

In [ ]:
# 读取
text_sz_2018_2020_path ='F:\\zzqaq\\data\\text_sz_2018_2020.csv'
text_sz_2018_2020 = pd.read_csv(text_sz_2018_2020_path,encoding="gb18030",converters={'code':str,'year':str})
# pdfcatalog_2021["code"] = [str(i).replace("\t","") for i in pdfcatalog_2021["code"]]

-----------------------------------------------------------------------  
### 查看现有文档  

In [7]:
# st标签
path_label='F:\zzqaq\data\\label_is_ST_2021.csv'
label = pd.read_csv(path_label,encoding="gbk",converters={'code':str, 'year':str})

# 上海18-20
path_text_sh_2018_2020 ='F:\\zzqaq\\data\\text_sh_2018_2020.csv'
text_sh_2018_2020 = pd.read_csv(path_text_sh_2018_2020,encoding="gb18030",converters={'code':str,'year':str})

# 深圳 18-20
text_sz_2018_2020_path ='F:\\zzqaq\\data\\text_sz_2018_2020.csv'
text_sz_2018_2020 = pd.read_csv(text_sz_2018_2020_path,encoding="gb18030",converters={'code':str,'year':str})

# 创业板18-20
text_szcy_2018_2020_path='F:\\zzqaq\\data\\text_szcy_2018_2020.csv'
text_szcy_2018_2020 = pd.read_csv(path,encoding="gb18030",converters={'code':str, 'year':str})

# 合并
text_all_18_20 = pd.concat([text_sh_2018_2020, text_sz_2018_2020,text_szcy_2018_2020])

# # 去重
text_all_18_20.drop_duplicates(subset=['code', 'year'], keep='first', inplace=True)

# 按code排序,x相同则year
text_all_18_20 = text_all_2018_2020.sort_values(by=['code','year'], ascending=True)
text_all_18_20 = text_all_18_20.reset_index(drop=True)

text_all_18_20.to_csv('F:\\zzqaq\\data\\text_all_18_20.csv',encoding="gb18030",index = False)

In [8]:
text_all_18_20_path ='F:\\zzqaq\\data\\text_all_18_20.csv'
text_all_18_20 = pd.read_csv(text_all_18_20_path,encoding="gb18030",converters={'code':str,'year':str})
# pdfcatalog_2021["code"] = [str(i).replace("\t","") for i in pdfcatalog_2021["code"]]
print(text_all_18_20.shape)
text_all_18_20.head(2)

(10815, 3)


,code,year,text
0,000004,2018,深圳中国农大科技股份有限公司年年度报告全文第四节经营情况讨论与分析一概述年度，公司实现主营业...
1,000004,2020,深圳国华网安科技股份有限公司年年度报告全文第四节经营情况讨论与分析一概述年是公司业务转型后的...
